# Comparing the performance of the full model and the Kallenbach model

Although we don't consider either model directly production ready, it's interesting to ask whether we've seen a significant change in performance.

In [1]:
## Every notebook should start with this header
# The next lines mean that changes in Python files are instantaneously applied
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path

# As a sanity check, print out the current working directory
print(f"Running in {Path('').absolute()}")

import cfspopcon
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import xarray as xr
from cfspopcon.unit_handling import magnitude_in_units, ureg

import extended_lengyel
plt.style.use('paper_plots.mplstyle')

Running in /Users/tbody/Projects/extended-lengyel/notebooks


In [2]:
kallenbach_algorithm = cfspopcon.Algorithm.get_algorithm("kallenbach_idl_translation")
rcc_algorithm = cfspopcon.Algorithm.get_algorithm("kallenbach_model")

ds_rcc = xr.Dataset(data_vars=extended_lengyel.read_config(
    filepath        = Path("config.yml"),
    elements        = ["base", "target_constraints"],
    keys            = rcc_algorithm.input_keys,
    allowed_missing = rcc_algorithm.default_keys
))
ds_rcc = cfspopcon.Algorithm.get_algorithm("initialize_kallenbach_model").update_dataset(ds_rcc)

ds_kallenbach_no_fast_neutrals = xr.Dataset(data_vars=extended_lengyel.read_config(
    filepath        = Path("config.yml"),
    elements        = ["base", "machine_geometry", "target_constraints", "fast_neutrals", "field_at_omp"],
    keys            = kallenbach_algorithm.input_keys,
    allowed_missing = kallenbach_algorithm.default_keys,
    overrides       = dict(fast_neutral_fraction = 0.0),
))
ds_kallenbach_no_fast_neutrals = cfspopcon.Algorithm.get_algorithm("initialize_kallenbach_idl_translation").update_dataset(ds_kallenbach_no_fast_neutrals)

In [3]:
import timeit

kallenbach_main_loop = cfspopcon.Algorithm.get_algorithm("run_kallenbach_idl_translation")
rcc_main_loop = cfspopcon.Algorithm.get_algorithm("run_kallenbach_model")

number_of_repeats_for_timing = 10

kallenbach_time_per_eval = timeit.timeit(lambda: kallenbach_main_loop.update_dataset(ds_kallenbach_no_fast_neutrals), number=number_of_repeats_for_timing) / number_of_repeats_for_timing
print(f"Kallenbach: {kallenbach_time_per_eval * 1e3:.3g}ms per loop")
for ode_method in ["RK45", "RK23", "DOP853", "Radau", "BDF", "LSODA"]:
    rcc_time_per_eval = timeit.timeit(lambda: rcc_main_loop.update_dataset(ds_rcc.update({"ode_method": ode_method})), number=number_of_repeats_for_timing) / number_of_repeats_for_timing
    print(f"RCC {ode_method}: {rcc_time_per_eval * 1e3:.3g}ms per loop (factor {kallenbach_time_per_eval / rcc_time_per_eval:.3g})")

Kallenbach: 308ms per loop


RCC RK45: 585ms per loop (factor 0.527)


RCC RK23: 247ms per loop (factor 1.25)


RCC DOP853: 276ms per loop (factor 1.11)


RCC Radau: 25.8ms per loop (factor 11.9)


RCC BDF: 23.3ms per loop (factor 13.2)
RCC LSODA: 18.2ms per loop (factor 17)
